##  Imports

In [47]:
import pandas as pd
import requests
import json


# Make a dataframe with Bitcoin and Dogecoin market value for last 30 days vs USD

In [93]:
url = 'https://api.coingecko.com/api/v3/coins/bitcoin/market_chart?vs_currency=usd&days=30'
response = requests.get(url).json()

url_doge = 'https://api.coingecko.com/api/v3/coins/dogecoin/market_chart?vs_currency=usd&days=30'
response_doge = requests.get(url_doge).json()

ds_doge = pd.DataFrame(response_doge)

ds = pd.DataFrame(response)
ds_doge.head()

,prices,market_caps,total_volumes
0,"[1564250568529, 0.0027871752976164287]","[1564250568529, 335890561.96682894]","[1564250568529, 44112808.41170882]"
1,"[1564254006749, 0.0028076762408327273]","[1564254006749, 334335193.45703876]","[1564254006749, 44390006.83088862]"
2,"[1564257694759, 0.002795199529679223]","[1564257694759, 336889404.24518985]","[1564257694759, 43927910.3284862]"
3,"[1564261223568, 0.002780781902830106]","[1564261223568, 335547234.04416144]","[1564261223568, 42390312.13389234]"
4,"[1564264803071, 0.0027969065202413782]","[1564264803071, 337395545.67515737]","[1564264803071, 42490019.73006458]"


In [94]:
ds = ds['prices'].apply(pd.Series)
ds_doge = ds_doge['prices'].apply(pd.Series)

ds_doge.head()

,0,1
0,1.564251e+12,0.002787
1,1.564254e+12,0.002808
2,1.564258e+12,0.002795
3,1.564261e+12,0.002781
4,1.564265e+12,0.002797


In [95]:
from datetime import datetime

ds.columns=['date', 'price']
ds['date'] = (pd.to_datetime(ds['date'], unit='ms'))

ds_doge.columns=['date', 'price']
ds_doge['date'] = (pd.to_datetime(ds_doge['date'], unit='ms'))

ds_doge.head()

,date,price
0,2019-07-27 18:02:48.529,0.002787
1,2019-07-27 19:00:06.749,0.002808
2,2019-07-27 20:01:34.759,0.002795
3,2019-07-27 21:00:23.568,0.002781
4,2019-07-27 22:00:03.071,0.002797


In [96]:
ds['date'] = ds['date'].dt.round('H')
ds_doge['date'] = ds_doge['date'].dt.round('H')
ds = ds.rename(columns={'price':'Bitcoin'})
ds_doge = ds_doge.rename(columns={'price':'Dogecoin'})
table = pd.merge(ds, ds_doge, on='date')
table.head()

,date,Bitcoin,Dogecoin
0,2019-07-27 18:00:00,9458.748969,0.002787
1,2019-07-27 19:00:00,9416.385314,0.002808
2,2019-07-27 20:00:00,9440.231282,0.002795
3,2019-07-27 21:00:00,9471.833699,0.002781
4,2019-07-27 22:00:00,9539.452875,0.002797


In [97]:
ds['date'] = ds['date'].dt.date
ds_doge['date'] = ds_doge['date'].dt.date

table_day = pd.merge(ds, ds_doge, on='date')
table_day.head()


,date,Bitcoin,Dogecoin
0,2019-07-27,9458.748969,0.002787
1,2019-07-27,9458.748969,0.002808
2,2019-07-27,9458.748969,0.002795
3,2019-07-27,9458.748969,0.002781
4,2019-07-27,9458.748969,0.002797


In [98]:
table_day_collapse = table_day.groupby('date', as_index=False).agg({'Bitcoin':'mean','Dogecoin':'mean'})
table_day_collapse

,date,Bitcoin,Dogecoin
0,2019-07-27,9462.380811,0.002798
1,2019-07-28,9516.777434,0.002806
2,2019-07-29,9556.353549,0.002816
3,2019-07-30,9533.064726,0.002790
4,2019-07-31,9830.911493,0.002833
5,2019-08-01,10065.183360,0.002863
6,2019-08-02,10446.422284,0.002900
7,2019-08-03,10750.478958,0.002928
8,2019-08-04,10777.797120,0.002912
9,2019-08-05,11604.992007,0.003036


# Top Ten Coins Report

In [99]:
# Bitcoins' high:
ds2 = table.sort_values(by=['Bitcoin'], ascending=False)
print(ds2[0:10])
# Per day average:
ds21 = table_day_collapse.sort_values(by=['Bitcoin'], ascending=False)
print(ds21[0:10])

                   date       Bitcoin  Dogecoin
232 2019-08-06 10:00:00  12177.562769  0.003067
231 2019-08-06 09:00:00  12162.438504  0.003051
230 2019-08-06 08:00:00  12124.570430  0.003060
229 2019-08-06 07:00:00  12092.368935  0.003061
294 2019-08-09 00:00:00  12000.033897  0.002981
261 2019-08-07 15:00:00  11988.956792  0.002987
260 2019-08-07 14:00:00  11978.281250  0.002981
268 2019-08-07 22:00:00  11956.670767  0.002980
295 2019-08-09 01:00:00  11950.706649  0.002958
274 2019-08-08 04:00:00  11940.033948  0.002994
          date       Bitcoin  Dogecoin
13  2019-08-09  11825.444802  0.002901
12  2019-08-08  11797.919446  0.002946
10  2019-08-06  11749.143921  0.003004
11  2019-08-07  11683.889048  0.002952
14  2019-08-10  11608.465513  0.002871
9   2019-08-05  11604.992007  0.003036
16  2019-08-12  11400.132646  0.002879
15  2019-08-11  11387.295657  0.002877
17  2019-08-13  11167.670861  0.002849
8   2019-08-04  10777.797120  0.002912


In [102]:
# Dogecoin high:
ds_doge2 = table.sort_values(by=['Dogecoin'], ascending=False)
print(ds_doge2[0:10])
# Per day average:
ds_doge21 = table_day_collapse.sort_values(by=['Dogecoin'], ascending=False)
print(ds21[0:10])

                   date       Bitcoin  Dogecoin
217 2019-08-05 19:00:00  11780.739666  0.003096
215 2019-08-05 17:00:00  11784.364288  0.003087
216 2019-08-05 18:00:00  11716.837442  0.003082
232 2019-08-06 10:00:00  12177.562769  0.003067
218 2019-08-05 20:00:00  11727.253168  0.003065
229 2019-08-06 07:00:00  12092.368935  0.003061
230 2019-08-06 08:00:00  12124.570430  0.003060
213 2019-08-05 15:00:00  11694.203690  0.003059
208 2019-08-05 10:00:00  11679.070983  0.003058
211 2019-08-05 13:00:00  11728.119800  0.003052
          date       Bitcoin  Dogecoin
13  2019-08-09  11825.444802  0.002901
12  2019-08-08  11797.919446  0.002946
10  2019-08-06  11749.143921  0.003004
11  2019-08-07  11683.889048  0.002952
14  2019-08-10  11608.465513  0.002871
9   2019-08-05  11604.992007  0.003036
16  2019-08-12  11400.132646  0.002879
15  2019-08-11  11387.295657  0.002877
17  2019-08-13  11167.670861  0.002849
8   2019-08-04  10777.797120  0.002912


In [85]:
url3 = 'https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=10&page=1&sparkline=false'
response3 = requests.get(url3).json()
dsnew = pd.DataFrame(response3)
dsnew.head(10)

dsnew1=dsnew[['id','name', 'current_price', 'current_price', 'market_cap', 'total_volume']]
dsnew1.head(10)

,id,name,current_price,current_price,market_cap,total_volume
0,bitcoin,Bitcoin,10303.910000,10303.910000,184394240638,22771222829
1,ethereum,Ethereum,188.560000,188.560000,20273721302,8011241770
2,ripple,XRP,0.269982,0.269982,11589651051,1425389669
3,bitcoin-cash,Bitcoin Cash,308.510000,308.510000,5545746559,1604102139
4,litecoin,Litecoin,73.470000,73.470000,4637975948,2926735484
5,binancecoin,Binance Coin,26.010000,26.010000,4041857116,221675488
6,eos,EOS,3.590000,3.590000,3659158938,2244788932
7,tether,Tether,0.998733,0.998733,3556064395,37394128201
8,bitcoin-cash-sv,Bitcoin SV,133.690000,133.690000,2401755683,395526550
9,cardano,Cardano,0.049954,0.049954,1553976731,160916251


## List the first 10 coingecko currencies in a DataFrame

In [ ]:
# See previous 
"""
url3 = 'https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=10&page=1&sparkline=false'
response3 = requests.get(url3).json()
dsnew = pd.DataFrame(response3)
dsnew.head(10)

dsnew1=dsnew[['id','name', 'current_price', 'current_price', 'market_cap', 'total_volume']]
dsnew1.head(10)
"""

In [83]:
# The first 10 entries
#ds3 = table.sort_values(by=['date'], ascending=True)
#ds3[0:10]

## Make a csv called 'top_ten_report.csv' and check that it worked

In [89]:
dsnew1.to_csv('dsnew1.csv')